# SousChef - End to End Pipeline_4

**Objective**: Our objective for this notebook is to filter the dataframe of 5000+ All Recipes recipes, topic modelled meal types, and predicted cuisines and return a recommended recipe to the user based on the inputs

In [26]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


import pickle

In [27]:
# df = pd.read_csv('datasets/final_recipes.csv')

In [28]:
# df.head()

## EDA

In [29]:
# df.columns

In [30]:
# df.drop(['title_ing_joined'], axis = 1, inplace = True)
# df = df[['title', 'time', 'meal_type', 'cuisine', 'instructions', 'ingredients', 'formatted_text', 'joined']]

In [31]:
# df.columns = ['title', 'time', 'meal_type', 'cuisine','instructions', 'ingredients', 'list_ingredients', 'ingred_string']
# df.head()

In [32]:
# df['id'] = np.random.randint(0,df.shape[0], df.shape[0])

In [33]:
# df.columns

In [34]:
# df = df[['id', 'title', 'time', 'meal_type', 'cuisine', 'instructions', 'ingredients',
#        'list_ingredients', 'ingred_string']]
# df.head()

## User Inputs

In [36]:
df = pd.read_csv('datasets/final_dataset.csv')

In [37]:
# Function to filter a dataframe based on a list of ingredients
# to check if there's an ingredient in a recipe of df_in
def filterbyIng(df_in, list_ing):
    '''function returns another dataframe that includes recipes with, at least, one component from list_ing
    the output dataframe contains at least one ingredient
    Input:
        df_in    : dataframe to examine
        list_ing : list of ingredients,e.g., ['cheese','lettuce','tomato']
    '''
    from itertools import combinations

    df_out = pd.DataFrame([],columns= df_in.columns)
    for each in list_ing:
        df_containing = df_in[df_in['ingred_string'].str.contains(each)]
        df_out = df_out.append(df_containing)
        df_out = df_out.drop_duplicates(subset='title')

    return df_out

In [38]:
user_ing = ['sugar', 'flour', 'milk']

In [39]:
filterbyIng(df, user_ing)

,id,title,time,meal_type,cuisine,instructions,ingredients,list_ingredients,ingred_string
2,729,Everything Deviled Eggs,35,maindish,british,Place eggs in a saucepan and cover with water....,"['6 large eggs', '2 tablespoons full-fat plain...","['everything', 'deviled', 'egg', 'egg', 'full'...",everything deviled egg egg full fat plain gree...
5,1319,Strawberry Bruschetta,15,breakfast,southern_us,Preheat your oven's broiler. Spread a thin lay...,"['24 slices French baguette', '1 tablespoon bu...","['strawberry', 'bruschetta', 'slice', 'french'...",strawberry bruschetta slice french baguette bu...
11,663,Annie's Fruit Salsa and Cinnamon Chips,45,dessert,mexican,"In a large bowl, thoroughly mix kiwis, Golden ...","['2 kiwis, peeled and diced', '2 Golden Delici...","['annie', 'fruit', 'salsa', 'cinnamon', 'chip'...",annie fruit salsa cinnamon chip kiwi golden de...
20,451,Brown Sugar Smokies,30,breakfast,southern_us,Preheat oven to 350 degrees F (175 degrees C)....,"['1 pound bacon', '1 (16 ounce) package little...","['brown', 'sugar', 'smokies', 'bacon', 'packag...",brown sugar smokies bacon package little smoki...
22,938,Sugar Coated Pecans,0,dessert,southern_us,Preheat oven to 250 degrees F (120 degrees C)....,"['1 egg white', '1 tablespoon water', '1 pound...","['sugar', 'coated', 'pecan', 'egg', 'white', '...",sugar coated pecan egg white water pecan white...
...,...,...,...,...,...,...,...,...,...
5029,1895,Halloween Candy Corn Jell-O® Shots,225,drinks,southern_us,Pour 3/4 cup boiling water into a small bowl. ...,"['1 1/2 cups boiling water, divided', '1 (3 ou...","['halloween', 'candy', 'corn', 'jell', 'shot',...",halloween candy corn jell shot boiling water d...
5033,4510,Quick Strawberry Oatmeal Breakfast Smoothie,10,breakfast,british,Blend oats and chia seeds together in a blende...,"['1/2 cup rolled oats', '1 teaspoon chia seeds...","['quick', 'strawberry', 'oatmeal', 'breakfast'...",quick strawberry oatmeal breakfast smoothie ro...
5035,2316,Superfood Chocolate Pudding Smoothie,5,breakfast,mexican,"Blend banana, avocado, soy milk, berries, coco...","['1 small banana, chopped', '1/2 avocado', '1/...","['superfood', 'chocolate', 'pudding', 'smoothi...",superfood chocolate pudding smoothie banana av...
5036,82,Strawberry Orange Coconut Smoothie,10,breakfast,southern_us,"Place the strawberries, orange, coconut milk, ...","['2 1/2 cups hulled strawberries', '1 orange, ...","['strawberry', 'orange', 'coconut', 'smoothie'...",strawberry orange coconut smoothie hulled stra...


In [60]:
# A function to find recipes with intersections of ingredients  (in df_in)
def intersect(df_in, list_ing):
    '''Function to return a dataframe containing recipes, ordered by the measure of intersections. e.g.,
    If 3 ingredients, then it respectively outputs recipes with 3-way intersection, 2-way, no intersection
    Input:
        df_in       : initial dataframe
        list_ingred : list of ingredients 
    '''
    from functools import reduce
    from itertools import combinations

    intersect_ids = []
    
    # Make a dictionary containing dataframes,w/ key:val => index: id's (in dataframe)
    
    dict_dfs = dict()
    for ind, ingred in enumerate(list_ing):
        
        # Build dataframe
        df_out = pd.DataFrame([],columns= df_in.columns)
        df_containing = df_in[df_in['ingred_string'].str.contains(ingred)]
        df_out = df_out.append(df_containing)
        
        # append ids of dataframe to the dictionary
        dict_dfs[ind] = df_out.id.values.tolist()
        
    print(f'1. Recipes with {list_ing[0]}: {len(dict_dfs[0])}, {list_ing[1]}: {len(dict_dfs[1])}, {list_ing[2]}: {len(dict_dfs[2])}')
    
    # First find id's with 3-way intersections
    ids_3way = list(set.intersection(*map(set, [dict_dfs[0],dict_dfs[1],dict_dfs[2]])))
    #print('2. Recipes using all 3 ingredients: ', len(ids_3way))
    df_dummy = pd.DataFrame([], columns=df_in.columns)
    df_3way = df_dummy.append(df_in[df_in.id.isin(ids_3way)])
    print('2. Recipes using all 3 ingredients: ', df_3way.shape[0])

    # Then find id's with 2-way intersections
    ids_2way = []
    for x,y in combinations([dict_dfs[0],dict_dfs[1],dict_dfs[2]], 2):
        ids_2way += list(set.intersection(*map(set, [x,y])))
        ids_2way = list(set(ids_2way))
    #print('3. Recipes using 2 of 3 ingredients: ', len(ids_2way))
    df_2way = df_dummy.append(df_in[df_in.id.isin(ids_2way)])
    print('3. Recipes using 2 of 3 ingredients: ', df_2way.shape[0])

    # Finally, add individual
    for ingred in list_ing:
        df_each = df_dummy.append(df_in[df_in['ingred_string'].str.contains(ingred)])
    df_each = df_each.drop_duplicates(subset='id')
    print('4. Recipes using 1 of 3 ingredients: ',df_each.shape[0])

    # concat all df's and
    df_intersect = df_3way.append(df_2way).append(df_each)
    df_intersect = df_intersect.drop_duplicates(subset='id')
    print('5. Total recipe collection: ', df_intersect.shape[0])

    return df_3way.reset_index(drop=True), df_2way.reset_index(drop=True), df_each.reset_index(drop=True), df_intersect.reset_index(drop=True)


In [61]:
# The final function
def outputRecipes(df_in):
    '''Function returns another dataframe of only relevant data
    Input:
        df_in   - dataframe of interest
    Output:
        df_out  - containing 'title','ingredient','instructions'
        links   - list of url's for each recipe
        images  - list of image_paths
    '''
    df_out = df_in[['title','ingredients','instructions', 'meal_type', 'cuisine']]

    return df_out

In [96]:
# user_ing = ['chicken', 'mushroom', 'lettuce']
user_ing = ['flour', 'sugar', 'egg']
#user_ing = ['garlic', 'cheese', 'onion']

In [97]:
# Find recipes with intersections
d3, d2, d1, d = intersect(df, user_ing)
# Output recipes
out3 =outputRecipes(d3)
out2 =outputRecipes(d2)
out1 =outputRecipes(d1)

print('\n')
print('SousChef found the following recipes that use all 3 ingredients: ')
print(out3.title.head(3),'\n')

print('SousChef found the following recipes that use 2 of 3 ingredients: ')
print(out2.title.head(3),'\n')

print('SousChef found the following recipes that use (at least) 1 of 3 ingredients: ')
print(out1.title.head(3), '\n')

print('---------------------------------------')

print('For your convenience, SousChef recommends you make: ')
print(out3.title[0], '\n')

print('Which is of the meal type {} and a {} cuisine! \n'.format(out3.meal_type[0], out3.cuisine[0]))

print('Here are the specific ingredients and instructions: ')
print(out3.ingredients[0], '\n')
print(out3.instructions[0], '\n')

1. Recipes with flour: 1325, sugar: 2061, egg: 1885
2. Recipes using all 3 ingredients:  1628
3. Recipes using 2 of 3 ingredients:  2625
4. Recipes using 1 of 3 ingredients:  1578
5. Total recipe collection:  1704


SousChef found the following recipes that use all 3 ingredients: 
0              Fried Mozzarella Puffs
1               Strawberry Bruschetta
2    Garlicky Appetizer Shrimp Scampi
Name: title, dtype: object 

SousChef found the following recipes that use 2 of 3 ingredients: 
0       Everything Deviled Eggs
1    Easy Rumaki with Pineapple
2        Fried Mozzarella Puffs
Name: title, dtype: object 

SousChef found the following recipes that use (at least) 1 of 3 ingredients: 
0    Everything Deviled Eggs
1     Fried Mozzarella Puffs
2    Artichoke Hearts Gratin
Name: title, dtype: object 

---------------------------------------
For your convenience, SousChef recommends you make: 
Fried Mozzarella Puffs 

Which is of the meal type maindish and a italian cuisine! 

Here are th